# import 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/MLMA/project/Code Continuous Data')

In [3]:
%load_ext autoreload
%autoreload 2
from stroke_probability_map import *

In [4]:
%load_ext autoreload
%autoreload 2
from stroke_probability_map_1 import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import numpy as np
import os
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

import pandas as pd

from scipy import signal

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data as Data

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim.lr_scheduler as lr_scheduler

from tqdm import tqdm

import pickle

# File and Path

File name:   
EOG_00x_xx_xxx.csv:   
 00x: participant's ID.    
 xx:  index of the recording time.    
 xxx: index of the corresponding eye movements.    
      - e.g., xxx is the index of a training word.   

In [6]:
result_path = '/content/drive/MyDrive/Colab Notebooks/MLMA/project/Code Continuous Data/Result'

In [7]:
data_path = '/content/drive/MyDrive/Colab Notebooks/MLMA/project/Data/continuous'

In [8]:
result_path_isolated = '/content/drive/MyDrive/Colab Notebooks/MLMA/project/Code Isolated Data/Result'

# Dataset and Dataloader

In [9]:
class MyDataset(Dataset):
  def __init__(self, ID_list, R_time_list, model_list, flatten, model_type, root_dir='/content/drive/MyDrive/Colab Notebooks/MLMA/project/Data/continuous/00'):
    self.data = []
    self.stroke_probability_map = []
    self.label = np.zeros((0,))
    for id_idx in range(len(ID_list)):
      file_list = os.listdir(root_dir + str(ID_list[id_idx]) +'/training/')
      for f_idx in range(len(file_list)):
        if int(file_list[f_idx][8:10]) in R_time_list:
          data = pd.read_csv(root_dir + str(ID_list[id_idx]) +'/training/' + file_list[f_idx],  header=None).to_numpy()
          #print(data.shape)
          data = signal.resample(data, 1024, axis=0)
          #print(data.shape)
          s = stroke_probability_map_1(data, model_list[id_idx], flatten, model_type)
          s = torch.from_numpy(s)
          self.stroke_probability_map.append(s)
          self.label = np.append(self.label, np.ones((1,))*(int(file_list[f_idx][-7:-4])-1), axis=0)
          data = (data - data.mean(axis=0, keepdims=True))/np.std(data,axis=0, keepdims=True)
          data = torch.from_numpy(data)
          self.data.append(data)

    self.label = torch.from_numpy(self.label)
  def __getitem__(self, index) :
    return self.data[index].float(), self.stroke_probability_map[index].float(), self.label[index].type(torch.LongTensor)
  def __len__(self):
    return len(self.data)

# Model

In [21]:
# 比1复杂
class My_CNN_2_split_B(nn.Module):
  def __init__(self, N=16):
    super(My_CNN_2_split_B, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=2, out_channels=16, kernel_size=9, padding=5)
    nn.init.xavier_uniform_(self.conv1.weight)
    nn.init.constant_(self.conv1.bias, 0.0)
    self.mp1 = nn.MaxPool1d(kernel_size=4)
    self.relu1 = nn.LeakyReLU()

    self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, padding=3)
    nn.init.xavier_uniform_(self.conv2.weight)
    nn.init.constant_(self.conv2.bias, 0.0)
    self.mp2 = nn.MaxPool1d(kernel_size=2)
    self.relu2 = nn.LeakyReLU()

    self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
    nn.init.xavier_uniform_(self.conv3.weight)
    nn.init.constant_(self.conv3.bias, 0.0)
    self.mp3 = nn.MaxPool1d(kernel_size=2)
    self.relu3 = nn.LeakyReLU()

    self.conv4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
    nn.init.xavier_uniform_(self.conv4.weight)
    nn.init.constant_(self.conv4.bias, 0.0)
    self.amp =  nn.AdaptiveMaxPool1d(N)
    self.sigmoid = nn.Sigmoid()
    

    self.fc1 = nn.Linear(int(128), 25)
    nn.init.xavier_uniform_(self.fc1.weight)
    nn.init.constant_(self.fc1.bias, 0.0)
    self.relu5 = nn.LeakyReLU()

    self.flatten = nn.Flatten()
    self.dropout = nn.Dropout(p=0.5)

    self.fc2 = nn.Linear(25*16, 150)
    nn.init.xavier_uniform_(self.fc2.weight)
    nn.init.constant_(self.fc2.bias, 0.0)

  def forward(self, x, probability):
    x = torch.transpose(x, -1, -2)

    x = self.relu1(self.mp1(self.conv1(x)))
    x = self.relu2(self.mp2(self.conv2(x)))
    x = self.relu3(self.mp3(self.conv3(x)))
    x = self.conv4(x)
    x = self.sigmoid(self.amp(x))
    x = torch.transpose(x, -1, -2)

    x = self.relu5(self.fc1(x))
    x = self.flatten(x)
    x = self.dropout(x)
    x = self.fc2(x)

    return x

In [20]:
# 比1复杂
class My_CNN_2_split_A(nn.Module):
  def __init__(self, N=16):
    super(My_CNN_2_split_A, self).__init__()
    self.fc1 = nn.Linear(int(3*12), 25)
    nn.init.xavier_uniform_(self.fc1.weight)
    nn.init.constant_(self.fc1.bias, 0.0)
    self.relu5 = nn.LeakyReLU()

    self.flatten = nn.Flatten()
    self.dropout = nn.Dropout(p=0.5)

    self.fc2 = nn.Linear(25*16, 150)
    nn.init.xavier_uniform_(self.fc2.weight)
    nn.init.constant_(self.fc2.bias, 0.0)

  def forward(self, x, probability):
    x = probability
    x = self.relu5(self.fc1(x))
    x = self.flatten(x)
    x = self.dropout(x)
    x = self.fc2(x)

    return x

# Some Funtions

In [12]:
def Train_model(model,epochs,optmizer,criterion, train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    train_loss = []
    x = 0
    for batch_idx, (data, probability, label) in enumerate(train_loader):
        data = data.to(device)
        label = label.to(device)
        probability = probability.to(device)
        optmizer.zero_grad()  
        output = model(data, probability)
        # print(type(output), type(label))
        loss = criterion(output, label)
        loss.backward()
        optmizer.step() 
        train_loss.append(loss.item()) 
        #if(batch_idx+1)%3 == 0:
            #print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.\
                #format(epochs, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))
    return np.mean(train_loss), model

In [13]:
def test(model,epochs,criterion, dataloader, T_T='Test'):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    test_loss = 0
    c = 0
    y_pred = np.zeros((0,))
    y_true = np.zeros((0, ))
    with torch.no_grad():
        for data, probability, label in dataloader:
            data = data.to(device)
            label = label.to(device)
            probability = probability.to(device)
            output = model(data, probability)
            test_loss += criterion(output, label).item()
            y_pred = np.append(y_pred, output.cpu().argmax(dim=-1), axis=0)
            y_true = np.append(y_true, label.cpu(), axis=0)
            c += 1
    test_loss /= c
    #if epochs%50 == 0:
        #print('EPOCH:{}    {} set: Average loss: {:.4f}'.format(epochs, T_T, test_loss))
  
    return test_loss, accuracy_score(y_true, y_pred), f1_score(y_true, y_pred, average='micro', zero_division=0), \
    f1_score(y_true, y_pred, average='macro', zero_division=0), precision_score(y_true, y_pred, average='micro', zero_division=0), \
    precision_score(y_true, y_pred, average='macro', zero_division=0), recall_score(y_true, y_pred, average='micro', zero_division=0), \
    recall_score(y_true, y_pred, average='macro', zero_division=0)

In [18]:
def main_function(total_epoch=100, dataloader_list=None, model_para=None, model_save=False, model_name=None, cross_val=True, weight_decay=1e-3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    if model_para==0:
      model = My_CNN_2_split_A().to(device)
    else:
      model = My_CNN_2_split_B().to(device)
        
    optmizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=weight_decay)  
    #optmizer = optim.SGD(model.parameters(), lr=1e-3, weight_decay=weight_decay) 
    scheduler = lr_scheduler.StepLR(optmizer, step_size=50, gamma=0.5, verbose=False) 
    criterion = nn.CrossEntropyLoss().to(device)  
    
    train_loss = np.zeros((total_epoch))
    score = np.zeros((len(dataloader_list), 8, total_epoch)) # loss, accuracy, f1 score*2, precision*2, recall*2
    
    for i in tqdm(range(total_epoch)):
        train_loss[i], model = Train_model(model, i, optmizer, criterion, dataloader_list[0])
        scheduler.step()
        for j in range(len(dataloader_list)):
            if j == 0:
                T_T = 'Train'
            elif j == 1:
                T_T = 'Validation'
            else:
                T_T = 'Test'
            score[j, :, i] = test(model, i, criterion, dataloader_list[j], T_T=T_T)
        #if np.argmin(score[1, 0, :(i+1)]) == i:
            #if model_save:
                #torch.save(model.state_dict(), model_name)     
    #if model_save: 
        #np.savez(model_name[:-4], train_loss=train_loss, score=score)
    if model_save:
      torch.save(model.state_dict(), model_name)
    if cross_val:
        epoch_best = np.argmin(score[1, 0, :])
        return score[:, :, epoch_best]
    else:
        return train_loss, score

## Leave one trial out split

In [22]:
BATCH_SIZE = 128
ID_list = [1,2,3,4,5,6]
model_name_list = ['mlp', 'SVC']
save_number = 0

for m_idx in range(1):
  model_list = []
  for ID in range(1, 7):
    model_name = result_path_isolated + '/'+ model_name_list[m_idx] + '_sub00' + str(ID) + '.pck'
    model_stroke = pickle.load(open(model_name, "rb"))
    model_list.append(model_stroke)

  score1 = np.zeros((4, 3, 8))
  score2 = np.zeros((4, 3, 8))
  for fold_idx in range(1, 5):
    test_dataset = MyDataset(ID_list, R_time_list=[fold_idx], model_list=model_list, flatten=True, model_type=0)
    test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    x = fold_idx+1 if fold_idx+1<=5 else 1
    val_dataset = MyDataset(ID_list, R_time_list=[x], model_list=model_list, flatten=True, model_type=0)
    val_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    R_time_list = []
    for i in range(1, 9):
      if i!=fold_idx and i!=x:
        R_time_list.append(i)
    train_dataset = MyDataset(ID_list, R_time_list=R_time_list, model_list=model_list, flatten=True, model_type=0)
    train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

      
    model_param=0
    score1[fold_idx-1] = main_function(total_epoch=200, dataloader_list=[train_loader, val_loader, test_loader], \
                                  model_para=model_param, model_save=False, cross_val=True, weight_decay=1e-4)
    print('A fold{} Train Loss{}, Train Acc{}'.format(fold_idx, score1[fold_idx-1, 0, 0], score1[fold_idx-1, 0, 1]))
    print('A fold{} Val Loss{}, Val Acc{}'.format(fold_idx, score1[fold_idx-1, 1, 0], score1[fold_idx-1, 1, 1]))
    print('A fold{} Test Loss{}, Test Acc{}'.format(fold_idx, score1[fold_idx-1, 2, 0], score1[fold_idx-1, 2, 1]))
    torch.cuda.empty_cache()

    model_param=1
    score2[fold_idx-1] = main_function(total_epoch=200, dataloader_list=[train_loader, val_loader, test_loader], \
                                  model_para=model_param, model_save=False, cross_val=True, weight_decay=1e-4)
    print('B fold{} Train Loss{}, Train Acc{}'.format(fold_idx, score2[fold_idx-1, 0, 0], score2[fold_idx-1, 0, 1]))
    print('B fold{} Val Loss{}, Val Acc{}'.format(fold_idx, score2[fold_idx-1, 1, 0], score2[fold_idx-1, 1, 1]))
    print('B fold{} Test Loss{}, Test Acc{}'.format(fold_idx, score2[fold_idx-1, 2, 0], score2[fold_idx-1, 2, 1]))
    torch.cuda.empty_cache()

  np.save(result_path + '/d0425_SPLIT_A_score_' + str(save_number), score1)
  save_number += 1
  np.save(result_path + '/d0425_SPLIT_B_score_' + str(save_number), score2)
  save_number += 1

  x = np.mean(score1, axis=0)
  print('Train', x[0])
  print('Val', x[1])
  print('Test', x[2])

  x = np.mean(score2, axis=0)
  print('Train', x[0])
  print('Val', x[1])
  print('Test', x[2])

100%|██████████| 200/200 [01:02<00:00,  3.21it/s]


A fold1 Train Loss0.2605916437777606, Train Acc0.9930555555555556
A fold1 Val Loss3.1531545221805573, Val Acc0.28888888888888886
A fold1 Test Loss3.3566650450229645, Test Acc0.28555555555555556


100%|██████████| 200/200 [01:19<00:00,  2.52it/s]


B fold1 Train Loss0.001067140787861056, Train Acc1.0
B fold1 Val Loss0.3925686225993559, Val Acc0.9033333333333333
B fold1 Test Loss0.34193918216715247, Test Acc0.9122222222222223


100%|██████████| 200/200 [01:04<00:00,  3.11it/s]


...

B fold3 Train Loss0.0009762187784707004, Train Acc1.0
B fold3 Val Loss0.33548669230003725, Val Acc0.9043381535038932
B fold3 Test Loss0.41508640348911285, Test Acc0.91


100%|██████████| 200/200 [01:04<00:00,  3.12it/s]


A fold4 Train Loss0.48054294423623517, Train Acc0.9762687112084703
A fold4 Val Loss3.070296809077263, Val Acc0.2661469933184855
A fold4 Test Loss2.982828974723816, Test Acc0.3114571746384872


100%|██████████| 200/200 [01:17<00:00,  2.60it/s]

B fold4 Train Loss0.0011854521237017418, Train Acc1.0
B fold4 Val Loss0.3057566001371015, Val Acc0.9142538975501113
B fold4 Test Loss0.3104901406913996, Test Acc0.9199110122358176
Train [0.38035393 0.98191523 0.98191523 0.98197705 0.98191523 0.98286258
 0.98191523 0.98206509]
Val [3.1633663  0.29078777 0.29078777 0.28503662 0.29078777 0.30399916
 0.29078777 0.29083333]
Test [3.19286969 0.30258652 0.30258652 0.29483052 0.30258652 0.31439798
 0.30258652 0.30255556]
Train [9.69838897e-04 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
Val [0.34743005 0.91048135 0.91048135 0.90989254 0.91048135 0.92014286
 0.91048135 0.9105    ]
Test [0.33783547 0.91581109 0.91581109 0.91579217 0.91581109 0.92727513
 0.91581109 0.91583333]
